# Pumped-Storage Optimisation with Genetic Algorithm and MILP

In [1]:
import pandas as pd
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product

# Import own implementations
from milp import MILP
import genetic
from genetic import GA_Actions_Elite, GA_Actions_Tournament

# Importing tuning libraries
import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler

background_colour = "#F2F2F2"
pn.theme_set(
    pn.theme_classic()
    + pn.theme(
        text=pn.element_text(family="monospace"),
        plot_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        panel_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        legend_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
    )
)

%load_ext blackcellmagic

## Reading the Price data

In [2]:
df = pd.read_csv("../01 - Data/example_week.csv")
df.head(2)

,spot,utc_time
0,101.54,2022-01-01 00:00:00+00:00
1,52.13,2022-01-01 01:00:00+00:00


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   spot      168 non-null    float64
 1   utc_time  168 non-null    object 
dtypes: float64(1), object(1)
memory usage: 2.8+ KB


## The Power Plant

In [4]:
plant_params = {
    "EFFICIENCY": 0.75,
    "MAX_STORAGE_M3": 5000,
    "MIN_STORAGE_M3": 0,
    "TURBINE_POWER_MW": 100,
    "PUMP_POWER_MW": 100,
    "TURBINE_RATE_M3H": 500,
    "MIN_STORAGE_MWH": 0,
    "INITIAL_WATER_LEVEL_PCT": 0,
}
plant_params["INITIAL_WATER_LEVEL"] = (
    plant_params["INITIAL_WATER_LEVEL_PCT"] * plant_params["MAX_STORAGE_M3"]
)
plant_params["PUMP_RATE_M3H"] = (
    plant_params["TURBINE_RATE_M3H"] * plant_params["EFFICIENCY"]
)
plant_params["MAX_STORAGE_MWH"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * plant_params["TURBINE_POWER_MW"]

## GA Actions

### Elite

In [11]:
ga_solver = GA_Actions_Elite(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

In [12]:
analysis = ga_solver.tune(
    tune_config={
        "MUTPB": 1,
        "POP_SIZE": tune.choice([100, 500, 5000]),
        "MUT_IND_PB": tune.uniform(0.01, 0.5),
        "ELITISM": tune.choice(np.arange(0.1, 0.7, 0.1)),
    },
    total_generations=100,
    timeout_s=60*45,
)

2024-04-01 19:44:25,018	INFO worker.py:1752 -- Started a local Ray instance.
2024-04-01 19:44:25,079	INFO packaging.py:530 -- Creating a file package for local directory '.'.
2024-04-01 19:44:25,110	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_147b67ec4bbe00e7.zip' (0.29MiB) to Ray cluster...
2024-04-01 19:44:25,113	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_147b67ec4bbe00e7.zip'.
2024-04-01 19:44:26,779	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
[I 2024-04-01 19:44:26,788] A new study created in memory with name: optuna


(train pid=20360) Selected: 150


Trial name,avg_fitness
train_074f0076,-9.07074e+06
train_1f88e6bb,-7.04066e+06
train_2c8210d4,-7.88221e+06
train_34dcb378,-7.31198e+06
train_39cc4391,-7.31973e+06
train_4659022a,-9.55388e+06
train_54b69b76,-7.33016e+06
train_57bdc3c9,-9.40161e+06
train_65c8bfb3,-7.84536e+06
train_8d7035c7,-1.00448e+07


(train pid=20360) Selected: 150
(train pid=20360) Selected: 150
(train pid=20360) Selected: 150
(train pid=20360) Selected: 150
(train pid=20360) Selected: 150
(train pid=20360) Selected: 150
(train pid=20360) Selected: 150
(train pid=20360) Selected: 150
(train pid=20360) Selected: 150 [repeated 5x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(train pid=20360) Selected: 150 [repeated 22x across cluster]
(train pid=20360) Selected: 150 [repeated 33x across cluster]
(train pid=20360) Selected: 150 [repeated 40x across cluster]
(train pid=27640) Selected: 50 [repeated 45x across cluster]
(train pid=27036) Selected: 60 [repeated 69x across cluster]
(train pid=27036) Selected: 60 [repeated 78x across cluster]
(train pid=27036) Selected: 60 [repeated 82x across cluster]
(train pid=27640) Selected: 50 [repeated 60x across cluste

(train pid=27640) C:\arrow\cpp\src\arrow\filesystem\s3fs.cc:2829:  arrow::fs::FinalizeS3 was not called even though S3 was initialized.  This could lead to a segmentation fault at exit


...

(train pid=30648) C:\arrow\cpp\src\arrow\filesystem\s3fs.cc:2829:  arrow::fs::FinalizeS3 was not called even though S3 was initialized.  This could lead to a segmentation fault at exit


(train pid=12160) Selected: 100 [repeated 57x across cluster]
(train pid=9128) Selected: 60 [repeated 57x across cluster]


2024-04-01 19:46:17,761	ERROR tune_controller.py:1332 -- Trial task failed for trial train_54b69b76
Traceback (most recent call last):
  File "c:\Users\mathi\miniconda3\envs\general\Lib\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "c:\Users\mathi\miniconda3\envs\general\Lib\site-packages\ray\_private\auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mathi\miniconda3\envs\general\Lib\site-packages\ray\_private\client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mathi\miniconda3\envs\general\Lib\site-packages\ray\_private\worker.py", line 2667, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users

(train pid=9128) Selected: 60 [repeated 53x across cluster]
(train pid=15876) Selected: 150 [repeated 25x across cluster]
(train pid=15876) Selected: 150 [repeated 20x across cluster]
(train pid=23796) Selected: 150 [repeated 24x across cluster]
(train pid=12940) Selected: 100 [repeated 26x across cluster]
(train pid=25776) Selected: 150 [repeated 30x across cluster]
(train pid=23796) Selected: 150 [repeated 26x across cluster]
(train pid=12940) Selected: 100 [repeated 30x across cluster]
(train pid=25776) Selected: 150 [repeated 26x across cluster]
(train pid=25776) Selected: 150 [repeated 26x across cluster]
(train pid=12940) Selected: 100 [repeated 30x across cluster]
(train pid=12940) Selected: 100 [repeated 27x across cluster]


2024-04-01 19:47:21,415	WARNING tune.py:229 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-04-01 19:47:21,462	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-04-01 19:47:21,498	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to 'C:/Users/mathi/ray_results/GA' in 0.0810s.
(train pid=15876) C

In [ ]:
analysis.best_config

In [ ]:
fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()),
    x="training_iteration",
    y="avg_fitness",
    color="trial_id",
)
fig.show()

In [ ]:
top_runs = analysis.dataframe().sort_values("avg_fitness", ascending=False).head(10)["trial_id"].to_list()

fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()).query(
        "trial_id in @top_runs"
    ),
    x="training_iteration",
    y="avg_fitness",
    color="trial_id",
)
fig.show()

In [ ]:
# (
#     analysis.dataframe()
#     .query("trial_id == 'd947e4c7'")
#     .filter(regex="config")
#     .melt()
#     .assign(variable=lambda x: x["variable"].str.replace("config/", ""))
#     .set_index("variable")
#     .to_dict()["value"]
# )

### Tournament

In [5]:
ga_solver = GA_Actions_Tournament(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

In [7]:
analysis = ga_solver.tune(
    tune_config={
        "MUTPB": 1,
        "POP_SIZE": tune.choice([100, 500, 5000]),
        "CXPB": tune.uniform(0.05, 0.95),
        "INITIAL_MUTATION_RATE": tune.uniform(0.02, 0.2),
        "FINAL_MUTATION_RATE": tune.uniform(0.005, 0.02),
        "TOURNAMENT_SIZE": tune.randint(1, 50),
    },
    total_generations=300,
    timeout_s=120*60,
)

2024-04-01 20:21:32,197	INFO worker.py:1585 -- Calling ray.init() again after it has already been called.
2024-04-01 20:21:32,201	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
[I 2024-04-01 20:21:32,208] A new study created in memory with name: optuna


Trial name,avg_fitness
train_250cc752,-9.98676e+06
train_4bb2f107,-9.99809e+06
train_7e3a051d,-9.90121e+06
train_c84fbb2b,-9.98514e+06
train_d224302a,-9.97546e+06
train_e04da3c7,-9.88841e+06
train_f8f0d94f,-9.99647e+06
train_f9d17d61,-9.99736e+06


In [ ]:
analysis.best_config

In [ ]:
fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()),
    x="training_iteration",
    y="avg_fitness",
    color="trial_id",
)
fig.show()

In [ ]:
top_runs = analysis.dataframe().sort_values("avg_fitness", ascending=False).head(10)["trial_id"].to_list()

fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()).query(
        "trial_id in @top_runs"
    ),
    x="training_iteration",
    y="avg_fitness",
    color="trial_id",
)
fig.show()

In [ ]:
(
    analysis.dataframe()
    .query("trial_id == 'd947e4c7'")
    .filter(regex="config")
    .melt()
    .assign(variable=lambda x: x["variable"].str.replace("config/", ""))
    .set_index("variable")
    .to_dict()["value"]
)